In [ ]:
import pandas as pd
import datetime
from datetime import timedelta

In [ ]:
df = pd.read_csv('order_brush_order.csv')
df.head()

# process
- turn id datatype to string
- find order brushing for each shop

In [ ]:
# adjust datatype
df['orderid'] = df['orderid'].astype(str)
df['shopid'] = df['shopid'].astype(str)
df['userid'] = df['userid'].astype(str)

In [ ]:
shopList = list(df['shopid'].unique())
shopN = len(shopList)
res = {}

for idx, shopid in enumerate(shopList):
    # inform
    print('start to check idx: %s, shopid: %s ...' % (idx, shopid), end='')
    
    # sort order by event time
    dfShop = df[df['shopid'] == shopid].sort_values('event_time', ascending=True)
    
    # tmp storage for order brushing user
    tmpRes = None
    
    # by shop, traverse order and check order brushing
    for orderid, _shopid, userid, event_time in dfShop.values:
        
        ### previous 1 hour (current order as 1 hour end)
        start_time = datetime.datetime.strptime(event_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-3600) # 1 hour
        tmpdf_ = dfShop[(dfShop['event_time'] >= str(start_time)) & (dfShop['event_time'] <= event_time)]
        
        # detect order brushing
        if len(tmpdf_)/len(tmpdf_['userid'].unique()) >= 3:
            if isinstance(tmpRes, pd.DataFrame):
                tmpRes = tmpdf_
            else:
                tmpRes = pd.concat([tmpRes, tmpdf_])
        else :
            continue
            
        ### next 1 hour (current order as 1 hour start)
        end_time = datetime.datetime.strptime(event_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=3600) # 1 hour
        tmpdf_ = dfShop[(dfShop['event_time'] >= event_time) & (dfShop['event_time'] < str(end_time))]
        
        # detect order brushing
        if len(tmpdf_)/len(tmpdf_['userid'].unique()) >= 3:
            if isinstance(tmpRes, pd.DataFrame):
                tmpRes = tmpdf_
            else:
                tmpRes = pd.concat([tmpRes, tmpdf_])
        else :
            continue
        
    # postprocess tmpRes to find suspicious buyer (or buyers)    
    if isinstance(tmpRes, pd.DataFrame):
        tmpRes = tmpRes.drop_duplicates()
        tmpRes = tmpRes.groupby('userid').count()
        ans = list(tmpRes[tmpRes['orderid'] == tmpRes['orderid'].max()].reset_index().sort_values('userid', ascending=True)['userid'].unique())
        res[shopid] = '&'.join(ans)
    else :
        res[shopid] = '0'
    
    # inform
    print('finish (%s / %s), ans : %s' % (idx+1, shopN, res[shopid]))


# Post-process
- create submission dataframe
- output as submission file

In [ ]:
keys = []
values = []
for key, value  in res.items():
    keys.append(key)
    values.append(value)

In [ ]:
resDf = pd.DataFrame({'shopid':keys, 'userid':values})
resDf.to_csv('submission.csv', index=False)